## 问题
你写了一个装饰器作用在某个函数上，但是这个函数的重要的元信息比如名字、文档字符串、注解和参数签名都丢失了。

## 解决方案
任何时候你定义装饰器的时候，都应该使用 **functools 库中的 @wraps** 装饰器来注解底层包装函数。例如：

In [1]:
import time
from functools import wraps
def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

下面我们使用这个被包装后的函数并检查它的元信息：

In [2]:
@timethis
def countdown(n):
    '''
    Counts down
    '''
    while n > 0:
     n -= 1

In [3]:
countdown(100000)

countdown 0.004056692123413086


In [4]:
countdown.__name__

'countdown'

In [5]:
countdown.__doc__

'\n    Counts down\n    '

In [6]:
countdown.__annotations__

{}

## 讨论
在编写装饰器的时候复制元信息是一个非常重要的部分。如果你忘记了使用 @wraps ， 那么你会发现被装饰函数丢失了所有有用的信息。比如如果忽略 @wraps 后的效果是下面这样的：

In [7]:
import time
from functools import wraps
def timethis1(func):
    '''
    Decorator that reports the execution time.
    '''
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [8]:
countdown.__name__

'countdown'

In [9]:
countdown(100000)

countdown 0.004998683929443359


In [10]:
countdown.__name__

'countdown'

In [11]:
countdown.__doc__

'\n    Counts down\n    '

In [12]:
@timethis1
def countdown(n):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1

In [13]:
countdown(100000)

countdown 0.003981828689575195


In [14]:
countdown.__name__

'wrapper'

In [15]:
countdown.__doc__

In [16]:
countdown.__annotations__

{}

@wraps 有一个重要特征是它能让你通过属性 __wrapped__ 直接访问被包装函数。例如:

In [18]:
countdown.__wrapped__(100000)

AttributeError: 'function' object has no attribute '__wrapped__'

In [19]:
@timethis
def countdown(n):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1

In [20]:
countdown.__wrapped__(100000)

__wrapped__ 属性还能让被装饰函数正确暴露底层的参数签名信息。例如：

In [21]:
from inspect import signature

In [22]:
print(signature(countdown))

(n)


一个很普遍的问题是怎样让装饰器去直接复制原始函数的参数签名信息， 如果想自己手动实现的话需要做大量的工作，最好就简单的使用 @wraps 装饰器。 通过底层的 __wrapped__ 属性访问到函数签名信息。更多关于签名的内容可以参考9.16小节。